# Predicting ship collisions in the arctic sea

In [180]:
#Import Libraries
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
import pyAgrum.lib.image as gumimage
import itertools


**Regarding the function below**<br>
The getIterations function takes in an array of items that would be the inputs to the "or" gate in the Fault Tree, The variable that would be the output to the fault tree and then the decision network itself

In [181]:
#Helper methods
def count(items):
    num = 0
    for i in items:
        if i == 1:
            num = num+1
    if num<len(items)/2: #Check if should be < or <=
        return True
    else:
        return False

def getIterations(vars, mainVariable, dn):
    lst = list(itertools.product([0, 1], repeat=len(vars)))
    items = {}
    for i in lst:
        for j in range(len(vars)):
            items[vars[j]] = i[j]
        #print(items)
        if count(i):
            #print("Method insert for", mainVariable, "dictionary is", items, "with probabilities [1,0]")
            dn.cpt(mainVariable)[items] = [1, 0]
        else:
            #print("Method insert for",mainVariable, "dictionary is", items, "with probabilities [0,1]")
            dn.cpt(mainVariable)[items] = [0, 1]

# Construction

## Define Nodes

In [182]:
#Define Nodes
dn = gum.InfluenceDiagram()
#Regular Nodes
var_names = ["RadarFailure", "Fog", "EnvironmentalObstacles", "Visibility", "HighSpeed", "HumanFactor", "Navigation", "EquipmentError", "DetectionFailure", 
             "RidgeIceAndIceberg", "PackIce", "HighWind", "Wave", "PackIceEffect", "NondetectedMultiLayerIce", "RidgeIceAndIcebergEffect", "WaveEffect", 
             "FaultOfOtherVessels", "IceBreakersFailure", "DangerousIceCondition", "TypeOfExpedition", "EnvironmentalOperationalEffects", "PotentialObstacles", "Collision"]

radarfailure, fog, environmentalobstacles, visibility, highspeed, humanfactor, navigation, equipmenterror, detectionfailure, ridgeiceandiceberg, packice, highwind, wave, packiceeffect, nondetectedmultilayerice, ridgeiceandicebergeffect, waveeffect, faultofothervessels, icebreakersfailure, dangerousicecondition, typeofexpedition, environmentaloperationaleffects, potentialobstacles, collision = [dn.add(gum.LabelizedVariable(name, name, 2)) for name in var_names]
#Decision Node
permit = dn.addDecisionNode(gum.LabelizedVariable("PermitExpedition", "PermitExpedition", 2))
#Utility Node
u = dn.addUtilityNode(gum.LabelizedVariable("U", "U", 1))
dn

(pyAgrum.InfluenceDiagram<double>@000002A2010358D0) Influence Diagram{
  chance: 24,
  utility: 1,
  decision: 1,
  arcs: 0,
  domainSize: 10^7.52575
}

## Arcs

Add in Arcs

In [183]:
dn.addArc(radarfailure, visibility)
dn.addArc(fog, visibility)
dn.addArc(environmentalobstacles, visibility)
dn.addArc(humanfactor, visibility)
dn.addArc(highspeed, navigation)
dn.addArc(humanfactor, navigation)
dn.addArc(equipmenterror, navigation)
dn.addArc(equipmenterror, detectionfailure)
dn.addArc(detectionfailure, ridgeiceandicebergeffect)
dn.addArc(ridgeiceandiceberg, ridgeiceandicebergeffect)
dn.addArc(packice, packiceeffect)
dn.addArc(highwind, packiceeffect)
dn.addArc(highwind, waveeffect)
dn.addArc(wave, waveeffect)
dn.addArc(packiceeffect, dangerousicecondition)
dn.addArc(nondetectedmultilayerice, dangerousicecondition)
dn.addArc(ridgeiceandicebergeffect, dangerousicecondition)
dn.addArc(waveeffect, environmentaloperationaleffects)
dn.addArc(navigation, environmentaloperationaleffects)
dn.addArc(visibility, environmentaloperationaleffects)
dn.addArc(faultofothervessels, potentialobstacles)
dn.addArc(icebreakersfailure, potentialobstacles)
dn.addArc(dangerousicecondition, potentialobstacles)
dn.addArc(environmentaloperationaleffects, collision)
dn.addArc(potentialobstacles, collision)
dn.addArc(typeofexpedition, permit)
dn.addArc(permit, u)
dn.addArc(collision, u)
gumimage.export(dn,"test_export.png")

## CPTs


Add in CPT <br>
Note: for "or" gates: B0 = B1+B2 <br>
Note: for "and" gates: B0 = B1*B2


For "Navigation" Effect I used the idea from this paper (https://reader.elsevier.com/reader/sd/pii/S0951832000000776?token=26EF05B58BEE12E1BFA7AF8C03ADFFF4715762438DDD0922D73E499AD84F80A194F925EA79A8B74025BECAE7F242470D&originRegion=eu-west-1&originCreation=20230504070556) as all gates represent causal relationships - therefore either true or false

In [184]:
dn.cpt("HumanFactor").fillWith([1-0.0076,0.0076])
dn.cpt("HighSpeed").fillWith([1-0.0068,0.0068])
dn.cpt("EquipmentError").fillWith([1-0.008563,0.008563])
#Get Navigation
vars = ["HumanFactor", "HighSpeed", "EquipmentError"]
getIterations(vars, "Navigation", dn)
dn.cpt("Navigation")

(pyAgrum.Potential<double>@000002A2026B4450) 
                    ||  Navigation       |
HighSp|HumanF|Equipm||0        |1        |
------|------|------||---------|---------|
0     |0     |0     || 1.0000  | 0.0000  |
1     |0     |0     || 1.0000  | 0.0000  |
0     |1     |0     || 1.0000  | 0.0000  |
1     |1     |0     || 0.0000  | 1.0000  |
0     |0     |1     || 1.0000  | 0.0000  |
1     |0     |1     || 0.0000  | 1.0000  |
0     |1     |1     || 0.0000  | 1.0000  |
1     |1     |1     || 0.0000  | 1.0000  |

CPT for "Visibility"

In [185]:
#Visibility impacted by RadarFailure, HumanFactor, EnvironmentalOnstacles and Fog
dn.cpt("RadarFailure").fillWith([1-0.0234, 0.0234])
dn.cpt("EnvironmentalObstacles").fillWith([1-0.4, 0.4])
dn.cpt("Fog").fillWith([1-0.120500, 0.120500])
vars = ["HumanFactor", "Fog", "EnvironmentalObstacles", "RadarFailure"]
getIterations(vars, "Visibility", dn)
dn.cpt("Visibility")

(pyAgrum.Potential<double>@000002A2026B3A30) 
                           ||  Visibility       |
RadarF|Fog   |Enviro|HumanF||0        |1        |
------|------|------|------||---------|---------|
0     |0     |0     |0     || 1.0000  | 0.0000  |
1     |0     |0     |0     || 1.0000  | 0.0000  |
0     |1     |0     |0     || 1.0000  | 0.0000  |
1     |1     |0     |0     || 1.0000  | 0.0000  |
0     |0     |1     |0     || 1.0000  | 0.0000  |
1     |0     |1     |0     || 1.0000  | 0.0000  |
[...4 more line(s) ...]
0     |1     |0     |1     || 1.0000  | 0.0000  |
1     |1     |0     |1     || 0.0000  | 1.0000  |
0     |0     |1     |1     || 1.0000  | 0.0000  |
1     |0     |1     |1     || 0.0000  | 1.0000  |
0     |1     |1     |1     || 0.0000  | 1.0000  |
1     |1     |1     |1     || 0.0000  | 1.0000  |

CPT for "WaveEffect"

In [186]:
dn.cpt("HighWind").fillWith([1-0.0093, 0.0093])
dn.cpt("Wave").fillWith([1-0.00953, 0.00953])
vars = ["HighWind", "Wave"]
getIterations(vars, "WaveEffect", dn)
dn.cpt("WaveEffect")

(pyAgrum.Potential<double>@000002A2026B47F0) 
             ||  WaveEffect       |
HighWi|Wave  ||0        |1        |
------|------||---------|---------|
0     |0     || 1.0000  | 0.0000  |
1     |0     || 1.0000  | 0.0000  |
0     |1     || 1.0000  | 0.0000  |
1     |1     || 0.0000  | 1.0000  |